# Tarea 2 :

JAVIERA KAID Y CLAUDIA DONOSO

- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada (valor de mercado significa el valor que hoy se puede observar.

In [3]:
#Paquetes
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

# from modules
import auxiliary as aux

from numpy import random as rnd
from scipy.stats import norm
import plotly.express as px
import pandas as pd
import numpy as np
import math

In [4]:
# Importe de data

frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
curva['t'] = curva['plazo'] / 365.0
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)
zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind='cubic',
                  fill_value="extrapolate")

In [5]:
# Funciones importadas de la clase

# calculo valor tasa
def zrate(t: float) -> float:
    return zcurva(t)

# calculo primera derivada


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta

# calculo segunda derivada


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

# calculo funcion f


def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

# primera derivada funcion f


def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

# calculo de theta


def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

# calculo de B


def b_hw(gamma: float, t: float, T: float) -> float:
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

# calculo de A


def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose=False):
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

# calculo de Z


def zero_hw(r: float, gamma: float, sigma: float,
            zrate: float, fwd: float, t: float, T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

# funcione de muchas simulaciones


def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed=None):

    dt = 1 / 264.0
    num_steps += 1

    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()

    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación.
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(1, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)

    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - \
                gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return sim

In [6]:
# DATOS

strike = 1
gamma = .5
sigma = .015
num_sim = 1000
num_step = 263
seed = 1234
r0 = zrate(0)
t = 0
T0 = 1
Tb = 2

In [7]:
# AD: Test

sim_hw_many(gamma, sigma, theta, r0, num_sim, num_step, seed)

array([[ 7.88140532e-04,  1.22187027e-03,  1.57987270e-04, ...,
         8.00667410e-03,  8.18170349e-03,  8.45569913e-03],
       [ 7.88140532e-04,  1.06274562e-03,  2.09250375e-03, ...,
        -2.89860134e-03, -3.00995617e-03, -2.97701244e-03],
       [ 7.88140532e-04,  2.23375524e-03,  1.69116792e-03, ...,
         6.34357483e-03,  6.52082819e-03,  6.95070150e-03],
       ...,
       [ 7.88140532e-04, -1.73057258e-05,  3.63043349e-05, ...,
        -7.68058972e-03, -7.73232186e-03, -8.01911308e-03],
       [ 7.88140532e-04,  1.05433302e-04,  1.83653095e-03, ...,
        -3.31355676e-03, -3.50778227e-03, -2.06946277e-03],
       [ 7.88140532e-04,  3.42583652e-04,  1.28897288e-03, ...,
         2.93160040e-03,  2.93765009e-03,  4.05419448e-03]])

In [8]:
#VALORIZACIÓN DE UNA OPCIÓN

In [9]:
# Funcion para valor teorico de una call y put sobre un bono cero cupon

def call_put(strike, r, gamma, sigma, zrate, fwd, t, T0, Tb) -> float:

    aux1 = (sigma**2)/(2*gamma)
    aux2 = 1-math.exp(-2*gamma*T0)
    Sz = b_hw(gamma,T0,Tb) * math.sqrt(aux1*aux2)
    z0 = zero_hw(r, gamma, sigma, zrate, fwd, t, T0)
    zb = zero_hw(r, gamma, sigma, zrate, fwd, t, Tb)
    aux1d1 = math.log(zb / (strike * z0)) # AD
    d1 = (aux1d1 + ((Sz ** 2)/2))/ (Sz)
    d2 = d1 - Sz
    call = z0 * ((zb/z0)* norm.cdf(d1) - strike * norm.cdf(d2))
    put = z0 * ( strike * norm.cdf(-d2) - (zb/z0) * norm.cdf(-d1))
    
    return call, put

In [27]:
# Funcion de valorizacion por montecarlo
def mc_opcion(strike, zrate, gamma, sigma, theta, r, num_sim, num_step, T0, Tb, fwd, seed):

    #df = curva['df'].iloc[15]
    precios = []
    call_sim = []
    put_sim = []
    df = 0
    dt = 1 / 264
    sim = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_step, seed)
    for s in sim:
        df += math.exp(-dt * np.sum(s)) # AD: Calculas un único factor de descuento y no
                                        # uno por trayectoria. Además el factor está mal calculado.
                                        # Estás haciendo el loop en todas las trayectorias.
    print('df: ', df)
    for i in range(num_sim):
        r = sim[i][-1]
        # print('r', r)
        precios.append(zero_hw(r, gamma, sigma, zrate, fwd, T0, Tb))
        # print('pi', precios[i])
        call_sim.append(max((precios[i] - strike), 0))
        # print(max((precios[i] - strike), 0))
        put_sim.append(max((strike - precios[i]), 0))
        # print(max((strike - precios[i]), 0))
        # print()
        

    precio_call = df * np.mean(call_sim) / len(sim) # AD: El promedio está calculado con np.mean
    precio_put = df * np.mean(put_sim) / len(sim)   # y luego vuelves a dividir.
                                                    # El efecto se netea con el df mal calculado.
    # print(type(df))
    # print(len(sim))
    # print(len(call_sim))
    return precio_call, precio_put

In [28]:
# comprobar la funcion call_put
(call, put) = call_put(strike, r0, gamma, sigma, zrate, fwd, t, T0, Tb)

# comprobar funcion de montecarlo
(c_sim, p_sim) = mc_opcion(strike, zrate, gamma, sigma,
                           theta, r0, num_sim, num_step, T0, Tb, fwd, seed)

# AD: mejoré el print. De esta forma es mucho más legible y fácil de corregir.
print(f'Valor teórico de una call: {call: .6},')
print(f'Valor por simulación de una call: {c_sim: .6},')
print(f'Error call: {abs(c_sim - call): .6%},')
print('-----------------------------------------------')
print(f'Valor teórico de una put: {put: .6},')
print(f'Valor por simulación de una put: {p_sim: .6},')
print(f'Error put: {abs(p_sim-put): .6%}')

df:  999.5921975190696
Valor teórico de una call:  0.00352839,
Valor por simulación de una call:  0.00345897,
Error call:  0.006942%,
-----------------------------------------------
Valor teórico de una put:  0.00396077,
Valor por simulación de una put:  0.00366192,
Error put:  0.029885%


**Test**

In [12]:
import hull_white as hw

In [13]:
# Los valores correctos son:
print(f'strike: {strike}')
call = hw.zcb_call_put(hw.CallPut.CALL, strike, T0, Tb,
                       zrate(0.0001), math.exp(-zrate(T0) * T0),
                       math.exp(-zrate(Tb) * Tb),
                       .5, .015)
print(f'call: {call:.6}')
put = hw.zcb_call_put(hw.CallPut.PUT, strike, T0, Tb,
                       zrate(0.0001), math.exp(-zrate(T0) * T0),
                       math.exp(-zrate(Tb) * Tb),
                       .5, .015)
print(f'put: {put:.6}')

strike: 1
call: 0.00352839
put: 0.00396077


In [14]:
# VALORIZACIÓN DE UN BONO CERO CUPÓN

In [15]:
def mc_bonozero (zrate, gamma, sigma, theta, r, num_sim, num_step, T0, Tb, fwd, seed ):
    
    dt = 1 / 264
    valores = 0
    sim = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_step, seed)
    for s in sim:
        valores += math.exp(-dt * np.sum(s))

    bono = valores / num_sim
    return  bono

In [16]:
#comprobar resultados

#valor teorico del bono cero cupon
df = curva['df'].iloc[15] #15 porque en esa casilla esta el plazo de 1Y

#valor simulado con montecarlo del bono
bono = mc_bonozero (zrate, gamma, sigma, theta, r0, num_sim, num_step, T0, Tb, fwd, seed )
print(f'Valor teórico bono cero cupon a 1Y: {df: .6},')
print(f'Valor por simulación del bono a 1Y: {bono: .6},')
print(f'Error del bono a 1Y: { abs(bono-df):.6%}')

Valor teórico bono cero cupon a 1Y:  0.999298,
Valor por simulación del bono a 1Y:  0.999592,
Error del bono a 1Y: 0.029433%


**AD:** muy bueno, ojalá hubieran puesto más casos en las opciones y en los factores de descuento.